In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import tellurium as te

import matplotlib.pyplot as plt

In [2]:
os.chdir('..')
os.getcwd()

'c:\\Users\\mkcoo\\Documents\\UW\\GitHub\\ECFERS'

In [3]:
data = pd.read_csv('kegg_labels.csv',dtype='str',encoding='us-ascii',encoding_errors='ignore')
data = data.where(data.notnull(), None)
translator_dict = pd.Series(data['Label'].values, index=data['KEGG ID']).to_dict()

In [4]:
metrics = pd.read_csv('results/coefficients/cleaned_metrics_7pert_240808.csv')
perts = pd.read_csv('concentration_df_7pert.csv')
sbm = pd.read_csv('src/frenda_brenda/Files/Reduced Complexity/SpeciesBaseMechanisms_VN_Metabolism_Gly_BIOSYN.csv')

In [5]:
malmet = metrics[metrics['Compound']=='C00149']

In [6]:
maltop = malmet.sort_values(by=['Total Production'], ascending=False).head(10)

In [12]:
maltop

,Perturbation ID,Compound,Final Concentration,Min Conc,Max Conc,Min Time,Max Time,Total Production,Production to Max,Half Max Time,Half Max Conc
224772,889,C00149,0.016170,0.000132,0.016170,57.657658,28800.000000,0.015818,0.015818,12021.621622,0.008255
104850,415,C00149,0.015545,0.000029,0.015545,115.315315,28800.000000,0.015193,0.015193,9282.882883,0.009601
35022,139,C00149,0.011193,0.000199,0.011193,28.828829,28800.000000,0.010842,0.010842,12684.684685,0.005769
224519,888,C00149,0.010588,0.000082,0.010588,86.486486,28800.000000,0.010237,0.010237,11906.306306,0.005469
82333,326,C00149,0.008752,0.000029,0.008752,115.315315,28800.000000,0.008401,0.008401,14385.585586,0.004554
303708,1201,C00149,0.007398,0.000029,0.007398,115.315315,28800.000000,0.007046,0.007046,20093.693694,0.003870
34769,138,C00149,0.007018,0.000129,0.007018,57.657658,28800.000000,0.006667,0.006667,11963.963964,0.003684
303455,1200,C00149,0.006304,0.000029,0.006304,115.315315,28800.000000,0.005952,0.005952,19920.720721,0.003331
112440,445,C00149,0.005605,0.000029,0.005605,115.315315,28800.000000,0.005253,0.005253,5909.909910,0.002981
238434,943,C00149,0.004734,0.000029,0.006557,115.315315,7726.126126,0.004382,0.006206,3113.513514,0.003462


In [7]:
base = perts.iloc[0]

In [8]:
pertsfilt = perts[perts['Perturbation ID'].isin(maltop['Perturbation ID'])]

In [9]:
def findDiff(base, pert):
    diff = pert != base
    different_columns = diff[diff].index.tolist()

    return different_columns[1]

In [10]:
for index, row in pertsfilt.iterrows():
    changedvar = findDiff(base, row)
    print(changedvar)
    if changedvar.startswith('v'):
        print(translator_dict[sbm[sbm['Relative']==changedvar]['Label'].values[0]])
    else:
        print(changedvar)

p_EC1319
p_EC1319
p_EC1319
p_EC1319
v4
NADPH
v18
3-Carboxy-1-hydroxypropyl-ThPP
v23
ATP
v97
Tetradecanoyl-ACP
v97
Tetradecanoyl-ACP
v106
Succinyl-CoA
v149
Acetyl-CoA
v149
Acetyl-CoA


In [13]:
heatmap = pd.read_csv('results/coefficients/240807_heatmap1.csv')

In [15]:
heatmap[heatmap['Unnamed: 0']=='C00149']

,Unnamed: 0,p_EC5319,p_EC27111,p_EC31311,p_EC41213,p_EC5311,p_EC2723,p_EC42111,p_EC2792,p_EC27140,...,v243,v244,v245,v246,v247,v248,v249,v250,v252,v253
108,C00149,-0.000628,-0.396972,-0.226781,0.001484,-0.000045,-0.000148,-0.00005,3.520364e-07,-0.000004,...,0.000034,-7.525917e-08,0.000752,-0.0,-2.060125e-08,-0.0,-0.0,-0.0,2.337114e-08,-0.0


In [21]:
heatmap.loc[108]

True

In [26]:
type(heatmap.loc[108][1])

C:\Users\mkcoo\AppData\Local\Temp\ipykernel_33780\435003441.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



numpy.float64

In [28]:
top10 = heatmap.drop(columns=heatmap.columns[0]).loc[108].dropna().abs().nlargest(10)

In [39]:
largest_10_with_sign = heatmap.drop(columns=heatmap.columns[0]).loc[108][top10.index]

result_df = pd.DataFrame({
    'ID': largest_10_with_sign.index,
    'coeff': largest_10_with_sign.values
})

In [40]:
result_df

,ID,coeff
0,v4,-3.120563
1,v97,1.110305
2,p_EC1319,0.968021
3,v110,0.808935
4,p_EC23316,0.784060
5,p_EC23139,-0.642972
6,v52,-0.601759
7,v85,-0.550278
8,v118,-0.514860
9,v64,-0.407849


In [42]:
for id in result_df['ID']:
    if id.startswith('v'):
        print(translator_dict[sbm[sbm['Relative']==id]['Label'].values[0]])
    else:
        print(id)

NADPH
Tetradecanoyl-ACP
p_EC1319
Isocitrate
p_EC23316
p_EC23139
S-Glutaryldihydrolipoamide-E
trans-Dodec-2-enoyl-ACP
Malonyl-[acyl-carrier protein]
Octanoyl-ACP


In [44]:
for c in result_df['coeff']:
    print(c)

-3.120562746013792
1.1103046951005655
0.9680214581990724
0.8089354013287913
0.7840595674556814
-0.6429720582530956
-0.6017587943061812
-0.5502777856247003
-0.5148602065236277
-0.4078487606834687


In [ ]:
for index, row in pertsfilt.iterrows():
    changedvar = findDiff(base, row)
    print(changedvar)
    if changedvar.startswith('v'):
        print(translator_dict[sbm[sbm['Relative']==changedvar]['Label'].values[0]])
    else:
        print(changedvar)